# Illustration of intrinsic dendritic filtering

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from brainsignals.plotting_convention import mark_subplots, simplify_axes

In [ ]:
d = 2 * 1e-6  # µm -> m
R_m = 3e4 * 1e-4 # Ohm cm² -> Ohm m²
R_i = 150 * 1e-2 # Ohm cm -> Ohm m
C_m = 1 * 1e-2  # uF / cm² -> F / m²
lambda_0 = np.sqrt(d * R_m / 4 / R_i)  # m

frequencies = np.arange(1501)
omegas = 2 * np.pi * frequencies
zs = np.linspace(0, 1.5e-3, 100)

tau = R_m * C_m

l = 1
s = np.sqrt(1 + 1j * omegas * tau)

def return_abs_H(s_, z_, l=1):
    exp_1 = 2 * s_ * l / lambda_0
    exp_2 = s_ * z_ / lambda_0
    
    if exp_1 > 30: 
        term1 = 0 + 0j
    else:
        num1 = np.exp(+exp_2)
        den1 = 1 + np.exp(exp_1)
        term1 = num1 / den1
        
    term2 = np.exp(-exp_2) / (1 + np.exp(-exp_1))
    H = np.pi * s_**2 * d / R_m * (term1 + term2)
    return np.abs(H)

def return_abs_H_matrix(s, z, l):
    H = np.zeros((len(z), len(s)), dtype=complex)
    for s_idx in range(len(s)):
        H[:, s_idx] = return_abs_H(s[s_idx], z, l)
    return H


fig = plt.figure(figsize=(2.5, 4.2))
fig.subplots_adjust(bottom=0.18, wspace=0.3, left=0.22, top=0.95)
ax1 = fig.add_subplot(211, xlabel="distance (mm)", xlim=[0, 1.5], 
                      ylabel=r"|${\bf \hat{i}}_{\rm{m}}$| (normalized)")
ax2 = fig.add_subplot(212, xlim=[0, 1000], xlabel="frequency (Hz)", 
                      ylabel=r"$\lambda_{\rm AC}$ (normalized)")

for f in [0, 100, 500, 1000, 1500]:
    f_idx = np.argmin(np.abs(frequencies - f))
    H = return_abs_H(s[f_idx], zs, l=1e9)
    ax1.plot(zs * 1e3, H / np.max(H), label="%d Hz" % f)
ax1.legend(frameon=False, ncol=1, loc=(0.6, 0.5))

lambda_ac_inf = lambda_0 * np.sqrt(2 / (1 + np.sqrt(1 + (omegas * tau)**2)))

ls = [0.1e-3, 0.5e-3, 1e-3]

for l in ls:
    z_ = np.linspace(0, l, 1000)
    H_abs = return_abs_H_matrix(s, z_, l)
    lambda_ac = np.abs(np.trapz(z_ * H_abs.T) / np.trapz(H_abs.T))
    ax2.plot(frequencies, lambda_ac / np.max(lambda_ac_inf), label="%1.1f mm" % (l*1e3))
    
ax2.plot(frequencies, lambda_ac_inf/ np.max(lambda_ac_inf), label="$\infty$")
ax2.legend(frameon=False)

simplify_axes([ax1, ax2])
mark_subplots([ax1, ax2], ["C", "D"], ypos=1.05)
fig.savefig("intrinsic_dend_filt.pdf")